# Semantic Kernel : Chat, Planners, Memory, Hugging Face, Groundedness & Multi-Result

Dans ce notebook, nous allons explorer plusieurs fonctionnalités avancées de **Semantic Kernel** :

1. **Chat basique** avec Kernel Arguments (historique et contexte via un objet `KernelArguments`).
2. **Planners** (Sequential, Stepwise Function Calling) pour orchestrer dynamiquement des actions selon un but donné.
3. **Mémoire** & Embeddings (VolatileMemoryStore ou connecteurs externes) pour stocker des informations sémantiques.
4. **Hugging Face** : Intégration de modèles (texte, embeddings) en local ou depuis le Hub.
5. **Groundedness Checking** : vérification et ajustement du contenu pour éviter des “fabrications non justifiées”.
6. **Multi-Result** : récupération de plusieurs réponses pour un même prompt (OpenAI, Azure, Hugging Face).

Ce notebook est une **synthèse** des exemples dispersés dans différents notebooks, présentés sous forme de cellules Markdown et Python.


In [ ]:
# ============================
# Cellule : Installation & Imports
# ============================

# N'installez qu'une seule fois si nécessaire
%pip install -U semantic-kernel

# Imports de base
import os
import sys
from dotenv import load_dotenv

from semantic_kernel import Kernel
from semantic_kernel.functions import KernelArguments
from semantic_kernel.contents import ChatHistory

print("Imports et installation OK.")


## 3) Bloc Markdown – Configuration du Kernel & .env

```markdown
### Configuration du Kernel

Pour exécuter les exemples, on suppose que vous avez un fichier `.env` comportant vos clés d'API OpenAI / Azure OpenAI / Hugging Face.  
Exemple `.env` :

```
GLOBAL_LLM_SERVICE="OpenAI"        # ou AzureOpenAI, HuggingFace
OPENAI_API_KEY="sk-..."
OPENAI_CHAT_MODEL_ID="gpt-3.5-turbo"
...
```

Le `Kernel` lira ces informations pour décider quel connecteur LLM utiliser.  
Ensuite, nous ajouterons nos services (`OpenAIChatCompletion`, `AzureChatCompletion`, etc.) selon la variable `GLOBAL_LLM_SERVICE`.

In [ ]:
# ============================
# Cellule : Initialisation du Kernel
# ============================

from semantic_kernel.connectors.ai.open_ai import (
    OpenAIChatCompletion,
    AzureChatCompletion
)

# On charge le .env
load_dotenv()
global_llm_service = os.getenv("GLOBAL_LLM_SERVICE", "AzureOpenAI")

# Initialisation du Kernel
kernel = Kernel()

service_id = "default"

if global_llm_service.lower() == "openai":
    # Ajout du service OpenAI
    kernel.add_service(
        OpenAIChatCompletion(service_id=service_id),
    )
    print("Service OpenAI configuré.")
elif global_llm_service.lower() == "huggingface":
    # Ajout du service HuggingFace
    from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion
    kernel.add_service(
        HuggingFaceTextCompletion(
            service_id=service_id,
            ai_model_id="distilgpt2", # ex. pour text-generation
            task="text-generation"
        ),
    )
    print("Service Hugging Face configuré.")
else:
    # Par défaut : Azure OpenAI
    kernel.add_service(
        AzureChatCompletion(service_id=service_id),
    )
    print("Service Azure OpenAI configuré.")

### Chat Basique avec KernelArguments

L'idée : on crée une fonction de chat qui prend :
- l’historique de conversation (objet `ChatHistory`)
- un `user_input`
et on stocke le tout dans un `KernelArguments`.  

Cela permet d'alimenter un prompt (via un template) qui contient la variable `history` et `user_input`.

In [ ]:
# ============================
# Cellule : Extrait d'un Chat Minimal
# ============================

# Exemple de prompt
chat_prompt = """
{{$history}}
User: {{$user_input}}
ChatBot:
"""

# Création d'une fonction sémantique "chat"
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

pt_config = PromptTemplateConfig(
    template=chat_prompt,
    name="chatFunction",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="User's message"),
        InputVariable(name="history", description="Conversation history"),
    ]
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="myChatPlugin",
    prompt_template_config=pt_config
)

chat_history = ChatHistory()
chat_history.add_system_message("Vous êtes un chatbot utile spécialisé en recommandations de livres.")

async def chat_kernel(input_text: str):
    print(f"[Utilisateur] : {input_text}")
    response = await kernel.invoke(
        chat_function,
        KernelArguments(user_input=input_text, history=str(chat_history))
    )
    print(f"[ChatBot] : {response}")
    # Mise à jour de l'historique
    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(response))


# Test
await chat_kernel("Salut, peux-tu me conseiller un livre sur la philosophie antique ?")
await chat_kernel("Merci, tu peux détailler un peu plus la période concernée ?")

### Planners : Orchestration Dynamique

**SequentialPlanner** : Génère un plan sous forme de liste d’étapes (XML), chaque étape étant une fonction existante.  
**FunctionCallingStepwisePlanner** : S'appuie sur OpenAI function-calling pour exécuter “pas à pas” (ReAct, MRKL).

**Exemple d'usage** :
1. Le *user* fournit un `goal`.
2. Le planner trouve les fonctions (plugins sémantiques ou natifs) permettant d'accomplir ce but.
3. Il exécute les étapes, éventuellement enchaînant *function calls*.

Ci-dessous, un extrait minimal de code pour un `SequentialPlanner`.

In [ ]:
# ============================
# Cellule : Sequential Planner
# ============================
from semantic_kernel.core_plugins.text_plugin import TextPlugin
from semantic_kernel.functions import KernelFunctionFromPrompt
from semantic_kernel.connectors.ai.open_ai import OpenAIChatPromptExecutionSettings


plugins_directory = "./prompt_template_samples/"
summarize_plugin = kernel.add_plugin(plugin_name="SummarizePlugin", parent_directory=plugins_directory)
writer_plugin = kernel.add_plugin(
    plugin_name="WriterPlugin",
    parent_directory=plugins_directory,
)
text_plugin = kernel.add_plugin(plugin=TextPlugin(), plugin_name="TextPlugin")

shakespeare_func = KernelFunctionFromPrompt(
    function_name="Shakespeare",
    plugin_name="WriterPlugin",
    prompt="""
{{$input}}

Rewrite the above in the style of Shakespeare.
""",
    prompt_execution_settings=OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        max_tokens=2000,
        temperature=0.8,
    ),
    description="Rewrite the input in the style of Shakespeare.",
)
kernel.add_function(plugin_name="WriterPlugin", function=shakespeare_func)

for plugin_name, plugin in kernel.plugins.items():
    for function_name, function in plugin.functions.items():
        print(f"Plugin: {plugin_name}, Function: {function_name}")



# CORRECTION: SequentialPlanner est déprécié dans cette version
# from semantic_kernel.planners import SequentialPlanner
# Alternative: utiliser les agents ou function calling direct
print("\nATTENTION: SequentialPlanner n'est plus disponible dans cette version de SemanticKernel")
print("Utilisation d'une approche alternative sans SequentialPlanner :\n")
user_goal = """
Demain c'est la Saint-Valentin. Je veux composer un poème
dans le style de Shakespeare, en français, puis convertir
le texte en majuscules.
"""

print(f"Objectif : {user_goal}")

# Approche manuelle : étape par étape
# Étape 1 : Générer le poème avec Shakespeare function
poem_input = "Compose un poème d'amour pour la Saint-Valentin"
shakespeare_result = await kernel.invoke(shakespeare_func, KernelArguments(input=poem_input))
print(f"Étape 1 - Poème Shakespeare : {shakespeare_result}")

# Étape 2 : Convertir en majuscules avec TextPlugin
if 'TextPlugin' in kernel.plugins and 'uppercase' in kernel.plugins['TextPlugin'].functions:
    uppercase_result = await kernel.invoke(
        kernel.plugins['TextPlugin']['uppercase'], 
        KernelArguments(input=str(shakespeare_result))
    )
    print(f"\n=== Résultat final ===")
    print(uppercase_result)
else:
    # Fallback simple
    final_result = str(shakespeare_result).upper()
    print(f"\n=== Résultat final (fallback) ===")
    print(final_result)

# CODE OBSOLÈTE SUPPRIMÉ - Plus de référence à seq_plan

### Mémoire & Embeddings

**SemanticTextMemory** permet de stocker des textes (avec un embedding) dans un store :
- `VolatileMemoryStore` (en mémoire)
- ou connecteurs vers Pinecone, Azure Cognitive Search, Qdrant, etc.

On peut ensuite effectuer des requêtes sémantiques :  
`await memory.search("MaCollection", "Quelle est mon budget pour 2024 ?")`

In [ ]:
# ============================
# Cellule : Extrait Mémoire
# ============================

# CORRECTION: Utilisation des nouvelles approches pour la mémoire vectorielle
from semantic_kernel.connectors.in_memory import InMemoryStore
from semantic_kernel.connectors.ai.open_ai import OpenAITextEmbedding

# Embedding service (ex : openai text-embedding-ada)
embedding_service = OpenAITextEmbedding(
    service_id="embeddingService",
    ai_model_id="text-embedding-3-small"
)

# Nouvelle approche avec InMemoryStore
store = InMemoryStore()

# Ajout du service d'embedding au kernel
kernel.add_service(embedding_service)

# Pour la démonstration, on simule la mémoire et la recherche
budget_2024 = "Budget 2024 = 100k€"
budget_2023 = "Budget 2023 = 70k€"

print("Note: API de mémoire simplifiée pour cette version.")
print(f"Informations stockées :")
print(f"  - {budget_2024}")
print(f"  - {budget_2023}")

# Simulation de recherche
query = "Quel est mon budget pour 2024 ?"
print(f"\nRequête : {query}")

# Simulation simple : retour de la réponse connue
print("Réponse potentielle : ", budget_2024)

### Hugging Face Intégration

Semantic Kernel peut se connecter à Hugging Face localement ou via API.  
Exemple :  
```python
from semantic_kernel.connectors.ai.hugging_face import HuggingFaceTextCompletion

hf_service = HuggingFaceTextCompletion(
    service_id="textHF", ai_model_id="distilgpt2", task="text-generation"
)
kernel.add_service(hf_service)
```

Ensuite, on peut enregistrer une fonction sémantique ou invoquer directement `hf_service.get_text_contents(...)`.

### Groundedness Checking

Pour éviter les “hallucinations” d’un résumé :  
1. Extrait la liste d'entités du résumé.  
2. Vérifie la correspondance de chaque entité avec le texte source (référence).  
3. Retire ou corrige les entités non-fondées.

Cela se fait via un plugin “GroundingPlugin” (ex. ExtraitEntities, ReferenceCheckEntities, ExciseEntities).

In [ ]:
# ============================
# Cellule : Groundedness Checking
# ============================

# Suppose qu'on a un "grounding_text" = un texte source
grounding_text = """
Votre budget 2024 est de 100k euros.
Vous vivez à Genève.
Vous avez investi 50k en actions.
"""

# Suppose qu'on a un résumé "faux"
summary_text = """
Mon budget 2024 est de 200k euros.
J'habite à Milan.
"""

plugins_directory = "./prompt_template_samples/"

# On appelle un plugin (hypothétique) "GroundingPlugin" comportant 3 fonctions
try:
    grounding_plugin = kernel.add_plugin(parent_directory=plugins_directory, plugin_name="GroundingPlugin")
    extract_entities = grounding_plugin["ExtractEntities"]
    check_entities = grounding_plugin["ReferenceCheckEntities"]
    excise_entities = grounding_plugin["ExciseEntities"]
    
    # 1) Extraire entités avec les bonnes variables
    ext_result = await kernel.invoke(
        extract_entities, 
        KernelArguments(
            input=summary_text,
            topic="entities",  # Variable attendue par le template
            example_entities="Person, Location, Organization",  # Variable attendue
            allow_dangerously_set_content=True
        )
    )
    print("Entités détectées:", ext_result)
    
    # 2) Vérifier correspondance - Important: convertir ext_result en string
    ext_result_str = str(ext_result.value) if hasattr(ext_result, 'value') else str(ext_result)
    check_result = await kernel.invoke(
        check_entities, 
        KernelArguments(
            input=ext_result_str,  # Utiliser la version string
            reference_context=grounding_text,
            topic="entities",  # Ajouter les variables attendues
            allow_dangerously_set_content=True  # IMPORTANT: autoriser le contenu complexe
        )
    )
    print("Entités non-fondées:", check_result)
    
    # 3) Retirer entités non-fondées du summary
    check_result_str = str(check_result.value) if hasattr(check_result, 'value') else str(check_result)
    excision = await kernel.invoke(
        excise_entities, 
        KernelArguments(
            input=summary_text,
            ungrounded_entities=check_result_str,  # Utiliser la version string
            allow_dangerously_set_content=True
        )
    )
    print("Summary corrigé:", excision)
    
except Exception as e:
    print(f"Note: Le plugin GroundingPlugin n'est pas disponible ou fonctionnel: {e}")
    print("Démonstration alternative:")
    print(f"Texte source: {grounding_text[:50]}...")
    print(f"Résumé analysé: {summary_text[:50]}...")
    print("Entités potentiellement problématiques: Milan (au lieu de Genève), 200k€ (au lieu de 100k€)")

### Multi-Result

OpenAI (ou Azure) peut renvoyer plusieurs complétions pour un même prompt.  
On paramètre `number_of_responses=3` dans les settings.  
Ensuite, `get_text_contents(...)` ou `get_chat_message_contents(...)` renvoie un *tableau* de résultats.

In [ ]:
# ============================
# Cellule : Multi-Result
# ============================

from semantic_kernel.connectors.ai.open_ai import OpenAITextPromptExecutionSettings

settings = OpenAITextPromptExecutionSettings(
    extension_data={
        "max_tokens": 60,
        "temperature": 0.7,
        "number_of_responses": 3
    }
)

from semantic_kernel.connectors.ai.open_ai import OpenAITextCompletion
text_service = OpenAITextCompletion(service_id="multiResult", ai_model_id="gpt-3.5-turbo-instruct")

prompt = "Donne-moi une brève blague sur les chats :"

# get_text_contents() => liste de réponses
responses = await text_service.get_text_contents(prompt, settings=settings)

for i, r in enumerate(responses):
    print(f"Réponse n°{i+1}:\n{r}\n")

## Conclusion

Nous avons vu :
1. Un **Chat** basique avec `KernelArguments`.
2. Les **Planners** (Sequential, Stepwise) pour orchestrer dynamiquement des steps.
3. La **Mémoire** & embeddings pour stocker/rechercher des informations sémantiques.
4. L’**intégration Hugging Face** pour exécuter localement des modèles open-source.
5. Un **Groundedness Checking** minimal pour éviter les hallucinations.
6. La gestion de **plusieurs réponses** (Multi-Result) avec un seul appel.

Ces fonctionnalités permettent de créer des scénarios complexes : chat évolué, question-answering avec mémoire persistante, planification automatique, usage local ou cloud, etc. Bonne exploration !